In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

2024-12-13 10:13:39.145205: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-13 10:13:39.402842: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-13 10:13:39.623153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734084819.819965      41 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734084819.874392      41 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 10:13:40.371325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Data Processing

This section outlines the steps involved in processing the data before further analysis or modeling.


1. General Data Processing.

2. Convert the following fields to lowercase for uniformity:
   - **Name**
   - **fieldlabel**

   This ensures that comparisons are case-insensitive.

## Grouping Forms

3. Group the forms based on:
   - **Form ID**(customform)
   - **name**


In [2]:

df = pd.read_csv('data/all_datav3.csv', delimiter=';', on_bad_lines='skip')
print(df)

#handle blocktype(removeit).
indices_to_drop = df[df['elementtype'] == 'block'].index
df = df.drop(indices_to_drop)

#put specialtype in place.
for index, row in df.iterrows():
    if row['elementtype'] == 'special' and pd.isnull(row['fieldtype']):  # Check if 'elementtype' is empty
        df.at[index, 'fieldtype'] = row['specialtype']  # Assign 'specialtype' value to 'elementtype'

df = df.drop(columns=['Kunde','fieldtype','specialtype','elementtype', 'customname','fieldobjectid','fieldparentid','fieldparenttype','blocktype','fieldrelation','language'])
df['fieldlabel'] = df['fieldlabel'].apply(str.lower)
df['name'] = df['name'].apply(str.lower)
data = df.groupby(['customform','name'])['fieldlabel'].apply(list).reset_index()
display(data)

            Kunde   customname  customform              name  fieldobjectid  \
0     abinventech      form876         876  Kundereklamation            877   
1     abinventech      form876         876  Kundereklamation            878   
2     abinventech      form876         876  Kundereklamation            879   
3     abinventech      form876         876  Kundereklamation            880   
4     abinventech      form876         876  Kundereklamation            881   
...           ...          ...         ...               ...            ...   
9446          ipw  form1683675     1683675   SI - Handleplan        1711069   
9447          ipw  form1683793     1683793  SI - Aktiviteter        1711096   
9448          ipw  form1683793     1683793  SI - Aktiviteter        1711098   
9449          ipw  form1683675     1683675   SI - Handleplan        1711129   
9450          ipw  form1683793     1683793  SI - Aktiviteter        1711192   

      fieldparentid    fieldparenttype elementtype 

,customform,name,fieldlabel
0,770,projektportefølje,"[projekt nr., oprettet den, oprettet af, proje..."
1,775,projektområde,[tekst]
2,876,claims,"[processor, description of claim, complaint ca..."
3,876,kundereklamation,"[vælg kunde, gadenavn, postnr, by, telefonnumm..."
4,876,reklamation,"[vælg kunde, e-mail, kontaktperson, behandles ..."
...,...,...,...
753,1683707,strategiske initiativer,"[tekst, mwb, id, periode]"
754,1683712,si - must win battles,[mwb]
755,1683736,si - perioder,[periode]
756,1683793,si - aktiviteter,"[aktivitet, deadline, status, superviserende l..."


In [3]:
dfVal = pd.read_csv('data/all_data-validationCSV.csv', delimiter=';', on_bad_lines='skip')
indices_to_drop2 = dfVal[dfVal['elementtype'] == 'block'].index
dfVal = dfVal.drop(indices_to_drop2)

#put specialtype in place.
for index, row in dfVal.iterrows():
    if row['elementtype'] == 'special' and pd.isnull(row['fieldtype']):  # Check if 'elementtype' is empty
        dfVal.at[index, 'fieldtype'] = row['specialtype']  # Assign 'specialtype' value to 'elementtype'

dfVal = dfVal.drop(columns=['Kunde','fieldtype','specialtype','elementtype', 'customname','fieldobjectid','fieldparentid','fieldparenttype','blocktype','fieldrelation','language'])
dfVal['fieldlabel'] = dfVal['fieldlabel'].apply(str.lower)
dfVal['name'] = dfVal['name'].apply(str.lower)
dataVal = dfVal.groupby(['customform','name'])['fieldlabel'].apply(list).reset_index()
display(dataVal)
#data.to_excel('validatedAllData.xlsx', index=False)

,customform,name,fieldlabel
0,493061,oprettelse af ipw.dk-site,"[server, manuelt tildeling af server, anmoder,..."
1,695920,budgettal,"[debiteret support, kursusvirksomhed]"
2,829798,lead til salg,"[ansvarlig, lead via, status fra salg, lead op..."
3,854278,normtid pr uge,"[udvikling ikke planlagt, sum]"
4,894000,reference demo,[oprettet af]
...,...,...,...
125,2584010,formularkatalog,"[oprettet af, oprettet, blank linje, seneste ã..."
126,2584018,formularkatalog - kategori,[kategori]
127,2584251,formularkatalog - ã˜vrige vurderinger,"[hvad gã¸r formularen god?, vurderet af, vurde..."
128,2584255,formularkatalog - vurderingsskala,[vurdering]


# Prepare Data for Training

## Create a Mapping Between Label and Index

1. **Label to Index Mapping**:  
   Create a mapping between the **field label** and a unique index. This allows us to efficiently handle categorical data during training.

## Create Encoding for "Field Label"

2. **Field Label Encoding**:  
   Encode the **"fieldlabel"** by counting how many times each label appears within a form. This gives us a numeric representation of label frequency in the form.

## Create Embedding for "Form Name"

3. **Form Name Embedding**:  
   Create an embedding for the **"form name"** using Tokenizer This will convert the form name into a dense vector representation.

## Pad the Embedding

4. **Padding the Embedding**:  
   To ensure uniformity in input length, **pad the embeddings** so that all embeddings have the same length. This is necessary for feeding the data into a model.

## Split the Data for Testing

5. **Testing Data Split**:  
   Split the dataset, keeping the last **100 forms** aside for testing purposes. This will allow us to evaluate the performance of the model on unseen data.



In [4]:

# Get all unique labels
all_labels = list(set(label for labels in data['fieldlabel'] for label in labels))

# Map labels to indices
label_to_index = {label: i for i, label in enumerate(all_labels)}
index_to_label = {i: label for label, i in label_to_index.items()}

# Total number of examples
total_samples = len(data)

# Split index (last 100 for testing, adjust for your data size)
split_index = max(0, total_samples - 100)

# Training and test split
train_data = data.iloc[:split_index]  # First part for training
test_data = data.iloc[split_index:]  # Last 100 for testing

# Prepare training data
X_train_names = train_data['name'].tolist()
Y_train_labels = np.zeros((len(train_data), len(all_labels)), dtype=np.float32)

#set one-hot encoding for field labels
for i, labels in enumerate(train_data['fieldlabel']):
    Y_train_labels[i, [label_to_index[label] for label in labels]] = 1

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_names) 
X_train_sequences = tokenizer.texts_to_sequences(X_train_names)
max_name_len = max(len(seq) for seq in X_train_sequences)  
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_name_len, padding='post')


# Prepare test data
X_test_names = test_data['name'].tolist()
Y_test_labels = np.zeros((len(test_data), len(all_labels)), dtype=np.float32)

for i, labels in enumerate(test_data['fieldlabel']):
    for label in labels:
        Y_test_labels[i, label_to_index[label]] = 1

X_test_sequences = tokenizer.texts_to_sequences(X_test_names)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_name_len, padding='post')


## Model Architecture

The model is composed of two main components:

### 1. LSTM (Sequence to Vector)

- **Purpose**: The first part of the model uses an **LSTM** (Long Short-Term Memory) network. This component is designed to process the sequence of embedded form names and convert them into a vector representation.
  
- **Input**: The input to the LSTM is the **embedded form name**, which has been preprocessed and embedded into a vector format.
  
- **Output**: The output of the LSTM is a dense vector that represents the form, capturing the sequence of the form name and its contextual information.

### 2. Multi-output Classifier

- **Purpose**: The second part of the model is a **multi-output classifier**. This classifier predicts the probability of the appearance of each label in a given form.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

vocab_size = len(tokenizer.word_index) + 1  # Include padding
embedding_dim = 128  # Dimension of embedding space
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

name_input = Input(shape=(max_name_len,))

# Embedding layer with masking enabled
x = Embedding(vocab_size, embedding_dim, mask_zero=True)(name_input)

# LSTM layer for sequence-to-vector encoding
x = LSTM(32, return_sequences=False)(x)

# Output layer for multi-label classification
output = Dense(len(all_labels), activation='sigmoid')(x)

# Build the model
model = Model(name_input, output)

# Compile the model with an optimizer and loss function
model.compile(optimizer= optimizer, loss='mse', metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='loss',
    patience=50,
    restore_best_weights=True
)

# Train the model
model.fit(X_train_padded, Y_train_labels, epochs=2000, batch_size=32, callbacks=[early_stopping])


2024-12-13 10:13:50.601268: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.0000e+00 - loss: 0.2186
Epoch 2/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0075 - loss: 0.0074    
Epoch 3/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0095 - loss: 0.0022    
Epoch 4/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0065 - loss: 0.0023    
Epoch 5/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0148 - loss: 0.0023    
Epoch 6/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.0309 - loss: 0.0023
Epoch 7/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0216 - loss: 0.0022
Epoch 8/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.0376 - loss: 0.0023    
Epoch 9/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.0203 - loss: 0.0021    
Epoch 10/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0028 - loss: 0.0022    
Epoch 11/1000
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0757 - loss: 0.0022
Epoch 12/

## Testing

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_padded, Y_test_labels, batch_size=32)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")



## Predictions
print out the labels with more that 40% chance to appeare

In [ ]:
# After training, predict using a test book name
form_name = "Sikkerhed"
name_seq = tokenizer.texts_to_sequences([form_name])
name_padded = pad_sequences(name_seq, maxlen=max_name_len, padding='post')

predictions = model.predict(name_padded)
predicted_labels = (predictions > 0.80).astype(int)  # Apply a threshold to get binary predictions

# Convert indices back to capture names
predicted_labels = [index_to_label[idx] for idx in np.where(predicted_labels[0] == 1)[0]]

print(f"Predicted Captures for '{form_name}': {predicted_labels}")

In [ ]:
# Convert indices back to capture names
testtest = [index_to_label[idx] for idx in X_test_padded.flatten()]
labeltest = [index_to_label[idx] for idx in Y_test_labels.flatten()]

maxPoints = 0;
overalPoints = 0;
negativePoints = 0;

# Grab form names of Validation Set
for row in dataVal.index:
    form_name = dataVal.loc[row, "name"]
    name_seq = tokenizer.texts_to_sequences([form_name])
    name_padded = pad_sequences(name_seq, maxlen=max_name_len, padding='post')
    predictions = model.predict(name_padded)
    predicted_labels = (predictions > 0.90).astype(int)  # Apply a threshold to get binary predictions
    # Convert indices back to capture names
    predicted_labels = [index_to_label[idx] for idx in np.where(predicted_labels[0] == 1)[0]]
    
    for label in dataVal.loc[row, "fieldlabel"]:
        maxPoints += 1;
        labels = label.split(",")
        for predLabel in predicted_labels:
            if(predLabel == label):
                overalPoints += 1;
                
    for label in predicted_labels:
        negativePoints += 1;


print(maxPoints)
print(overalPoints)
print(negativePoints)
percentage = overalPoints/maxPoints;
precision = overalPoints / (overalPoints + negativePoints)
precision = precision * 100;
percentage = percentage * 100;
print(f"Precision:'{precision}'")
print(f"percentage:'{percentage}'")
#print(testtest)
#print(labeltest)

In [ ]:
def SetupDataForPrediction(data):
    name_seq = tokenizer.texts_to_sequences([data])
    name_padded = pad_sequences(name_seq, maxlen=max_name_len, padding='post')
    return name_padded

In [ ]:

from flask import Flask, request, jsonify
import numpy as np
import joblib  # or pickle if you use pickle
from joblib import dump

dump(model, 'formModel.pkl')
from flask_cors import CORS

In [ ]:


app = Flask(__name__)
CORS(app) 


# Load the model and print it
try:
    modelv2 = joblib.load("formModel.pkl")
    print("Model loaded successfully:", modelv2)
except Exception as e:
    print("Error loading model:", e)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        print("Model loaded successfully:", modelv2)
        # Get input data from the request
        input_data = request.json['data']  # Assuming data comes in JSON format
        print(input_data)

        input_data = SetupDataForPrediction(input_data)
        # Predict using the model
        try:
            prediction = modelv2.predict(input_data)
            print("Raw prediction output:", prediction)
        except Exception as e:
            print("Error during prediction:", str(e))
            import traceback
            print("Traceback:", traceback.format_exc())
            return jsonify({'error': 'Prediction failed. Check input data or model.'}), 500
            

        predicted_labels = (prediction > 0.9).astype(int)  # Apply a threshold to get binary predictions

        # Convert indices back to capture names
        predicted_labels = [index_to_label[idx] for idx in np.where(predicted_labels[0] == 1)[0]]
        
        print(f"Predicted Captures for '{form_name}': {predicted_labels}")
        
        # Return prediction as JSON
        return jsonify({'prediction': predicted_labels})
    except Exception as e:
        return jsonify({'error': str(e)})

In [ ]:
from werkzeug.serving import run_simple
# Use run_simple to run Flask
run_simple('0.0.0.0', 5000, app)

172.18.0.1 - - [13/Dec/2024 10:33:20] "OPTIONS /predict HTTP/1.1" 200 -


Model loaded successfully: <Functional name=functional, built=True>
ny medarbejder
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step


172.18.0.1 - - [13/Dec/2024 10:33:21] "POST /predict HTTP/1.1" 200 -


Raw prediction output: [[2.6760048e-03 7.1651343e-04 2.8134245e-04 ... 1.5789828e-03
  2.8726532e-05 9.4599620e-04]]
Predicted Captures for 'vidensdeling': ['tidligere lønnummer', 'ansættelse ophører', 'evt. kaldenavn', 'mailadresse', 'udstedt', 'statsborgerskab', 'hent fra uopfordret ansøgning', 'ansat til', 'andre certifikater', 'overordnet', 'fastnetnummer', 'blank 15', 'postnummer', 'fødselsdato', 'gyldigt indtil', 'ansvarlig', 'bukser, jakke/kedeldragt', 'sidste gennemførte uddannelse (ikke årstal)', 'telefonnummer', 'registreringsnummer', 'deadline for gennemførelse', 'adresse', 'førstehjælpscertifikat', 'upload dokumentation', 'brug af sap eller tilsvarende programmer', 'bemærk! følgende informationer er fortrolige og skal behandles med største forsigtighed', 'gyldig til', 'brug af microsoft office (outlook, word, excel m.m.)', 'brandbekæmpelse', 'blank 15-0', 'initialer', 'tlf.nr. til pårørende', 'brug af smartphone', 'arbejdstelefon', 'andre sprog', 'efternavn', 'vigtigt! hvis